In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

#### 1. 将输入转化成TFRecord格式并保存。

In [2]:
# 生成整数型的属性
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# 生成字符串型的属性
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# 读取mnist数据。
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)
mnist = input_data.read_data_sets("../datasets/MNIST_data",dtype=tf.uint8, one_hot=True)
tf.logging.set_verbosity(old_v)

images = mnist.train.images
# 训练数据所对应的正确答案，可以作为一个属性保存在TFRecord中
labels = mnist.train.labels
# 训练数据的图像分辨率，可以作为example中的一个属性
pixels = images.shape[1]
num_examples = mnist.train.num_examples

# 输出TFRecord文件的地址。
filename = "Records/output.tfrecords"
# 创建一个writer来写TFRecord
writer = tf.python_io.TFRecordWriter(filename)
for index in range(num_examples):
    # 将图像矩阵转化成一个字符串
    image_raw = images[index].tostring()
    # 将一个样例转化为Example Protocol Buffer,并将所有信息写入这个结构
    example = tf.train.Example(features=tf.train.Features(feature={
        'pixels': _int64_feature(pixels),
        'label': _int64_feature(np.argmax(labels[index])),
        'image_raw': _bytes_feature(image_raw)
    }))
    # 将一个example写入TFRecord文件
    writer.write(example.SerializeToString())
writer.close()
print("TFRecord文件已保存。")

Extracting ../datasets/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../datasets/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../datasets/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../datasets/MNIST_data/t10k-labels-idx1-ubyte.gz
TFRecord文件已保存。


#### 2. 读取TFRecord文件

In [3]:
# 创建一个reader来读取TRRecord文件中的样例
reader = tf.TFRecordReader()
# 创建一个队列来维护输入文件列表
filename_queue = tf.train.string_input_producer(["Records/output.tfrecords"])
# 从文件中读取一个样例
# 也可使用read_up_to函数一次性读取多个样例
_,serialized_example = reader.read(filename_queue)

# 解析读取的样例
# 如果需要解析多个样例，可以使用parse_example函数
features = tf.parse_single_example(
    serialized_example,
    features={
        # tf提供两种不同的属性解析方法：
        # 1. tf.FixedLenFeature，解析结果为一个Tensor
        # 2. tf.VarLenFeature，解析结果为SparseTensor，用于处理稀疏矩阵
        'image_raw':tf.FixedLenFeature([],tf.string),
        'pixels':tf.FixedLenFeature([],tf.int64),
        'label':tf.FixedLenFeature([],tf.int64)
    })

# tf.decode_raw可以将字符串解析成图像对应的像素数组
images = tf.decode_raw(features['image_raw'],tf.uint8)
labels = tf.cast(features['label'],tf.int32)
pixels = tf.cast(features['pixels'],tf.int32)

sess = tf.Session()

# 启动多线程处理输入数据。
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess,coord=coord)

for i in range(10):
    image, label, pixel = sess.run([images, labels, pixels])
    print(label, pixel)

7 784
3 784
4 784
6 784
1 784
8 784
1 784
0 784
9 784
8 784
